In [1]:
import re 
import numpy as np
import pandas as pd

In [2]:
# Importing the dataset :
DATASET_COLUMNS=['target','ids','date','flag','user','text']
DATASET_ENCODING = "ISO-8859-1"
data = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', 
                 encoding=DATASET_ENCODING, 
                 names=DATASET_COLUMNS)

# Display of the first 5 lines :
data.sample(5)

,target,ids,date,flag,user,text
1219844,4,1989991038,Mon Jun 01 03:30:49 PDT 2009,NO_QUERY,alice_chee,afterellen is awesome sauce awesome interview...
781092,0,2323330456,Thu Jun 25 00:24:11 PDT 2009,NO_QUERY,JoshwaActon,@billythekid askd around mate and no one knows...
753402,0,2286873427,Mon Jun 22 17:02:35 PDT 2009,NO_QUERY,memily,JB Hi-Fi Carindale has just sold out of 50/1.8...
265914,0,1988891153,Sun May 31 23:41:01 PDT 2009,NO_QUERY,GinnyMcQueen,I hate when Rob has to go to bed early.
1423978,4,2058794445,Sat Jun 06 15:38:27 PDT 2009,NO_QUERY,tinayayo,http://twitpic.com/6s5ks - Good day in long be...


In [3]:
data.head()


,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
data.columns


Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [5]:
print('length of data is', len(data))


length of data is 1600000


In [6]:
data.shape

(1600000, 6)

In [8]:
# Replacing the values to ease understanding :
sub_data = data[['text','target']]

In [11]:
sub_data['text'].tail()


1599995    Just woke up. Having no school is the best fee...
1599996    TheWDB.com - Very cool to hear old Walt interv...
1599997    Are you ready for your MoJo Makeover? Ask me f...
1599998    Happy 38th Birthday to my boo of alll time!!! ...
1599999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: text, dtype: object

In [12]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
nltk.download('wordnet', "/kaggle/working/nltk_data/")
nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
nltk.data.path.append("/kaggle/working/nltk_data/")

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data/...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /kaggle/working/nltk_data/...
Archive:  /kaggle/working/nltk_data/corpora/wordnet.zip
   creating: /kaggle/working/nltk_data/corpora/wordnet/
  inflating: /kaggle/working/nltk_data/corpora/wordnet/lexnames  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adv  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.verb  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/data.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/index.adj  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/nltk_data/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/nltk_data/c

In [22]:
def process_text(text):
    text = re.sub(r'\s+', ' ', text, flags=re.I) # Remove extra white space from text
    
    text = re.sub(r"https?://\S+|www\.\S+"," ",text)
    
    # Removing html tags
    text = re.sub(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});"," ",text)

    text = re.sub(r'\W', ' ', str(text)) # Remove all the special characters from text

    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # Remove all single characters from text

    text = re.sub(r'[^a-zA-Z\s]', '', text) # Remove any character that isn't alphabetical

    text = text.lower()

    words = word_tokenize(text)

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    stop_words = set(stopwords.words("english"))
    Words = [word for word in words if word not in stop_words]

    Words = [word for word in Words if len(word) > 3]

    indices = np.unique(Words, return_index=True)[1]
    cleaned_text = np.array(Words)[np.sort(indices)].tolist()

    return cleaned_text

In [23]:
x=sub_data.drop('target',axis=1)
y=sub_data.target

In [24]:
texts=list(x['text'])


In [25]:
tokenize_text = [process_text(text) for text in texts]


In [26]:
print(tokenize_text[:10])


[['switchfoot', 'awww', 'bummer', 'shoulda', 'david', 'carr', 'third'], ['upset', 'update', 'facebook', 'texting', 'might', 'result', 'school', 'today', 'also', 'blah'], ['kenichan', 'dived', 'many', 'time', 'ball', 'managed', 'save', 'rest', 'bound'], ['whole', 'body', 'feel', 'itchy', 'like', 'fire'], ['nationwideclass', 'behaving'], ['kwesidei', 'whole', 'crew'], ['need'], ['loltrish', 'long', 'time', 'rain', 'fine', 'thanks'], ['tatianak', 'nope'], ['twittera', 'muera']]


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tokenize_text, y, test_size=0.2, random_state=42)

In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_vocab = 20000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(X_train)
word_idx = tokenizer.word_index  # Corrected syntax for accessing word index
v = len(word_idx)
print("the size of vocab =", v)  # Corrected spacing
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

2024-04-30 19:16:32.770978: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-30 19:16:32.771088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-30 19:16:32.903465: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


the size of vocab = 482085


In [32]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 100
X_train = pad_sequences(X_train,maxlen=maxlen)
X_test = pad_sequences(X_test,maxlen=maxlen)

In [33]:
from keras.models import Sequential
from keras.layers import Embedding,Dense,Input,GlobalMaxPooling1D,Dropout,Bidirectional
from tensorflow.keras.models import Model
from keras import optimizers
import numpy as np
D=100
inputt=Input(shape=(maxlen,))

# Model1

In [35]:
from tensorflow.keras.layers import Input, Embedding, Dropout, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

learning_rate = 0.0001  # Set your desired learning rate here
x = Embedding(v + 1, D)(inputt)
x = Dropout(0.5)(x)

# Apply convolutional layers
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(pool_size=4)(x)

x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(pool_size=4)(x)

# Flatten the output of the convolutional layers
x = Flatten()(x)

x = Dense(32, activation='relu')(x)
x = Dense(2, activation='softmax')(x)

# Define the model
model_1 = tf.keras.Model(inputs=inputt, outputs=x)

# Define optimizer with specified learning rate
optimizer = Adam(learning_rate=learning_rate)

model_1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
print(model_1.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 100, 100)       │    48,208,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 96, 128)        │        64,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 24, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 20, 64)         │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 320)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │        10,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 48,324,090 (184.34 MB)

 Trainable params: 48,324,090 (184.34 MB)

 Non-trainable params: 0 (0.00 B)

None


In [37]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [38]:
import tensorflow as tf

y_train_one_hot = tf.keras.utils.to_categorical(y_train_encoded)
y_test_one_hot = tf.keras.utils.to_categorical(y_test_encoded)
result_1 = model_1.fit(X_train, y_train_one_hot, epochs=5, validation_data=(X_test, y_test_one_hot))

Epoch 1/5
   19/40000 ━━━━━━━━━━━━━━━━━━━━ 5:58 9ms/step - accuracy: 0.5151 - loss: 0.6935

I0000 00:00:1714504935.260249     314 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40000/40000 ━━━━━━━━━━━━━━━━━━━━ 370s 9ms/step - accuracy: 0.7037 - loss: 0.5505 - val_accuracy: 0.7633 - val_loss: 0.4874
Epoch 2/5
40000/40000 ━━━━━━━━━━━━━━━━━━━━ 365s 9ms/step - accuracy: 0.7662 - loss: 0.4831 - val_accuracy: 0.7665 - val_loss: 0.4842
Epoch 3/5
40000/40000 ━━━━━━━━━━━━━━━━━━━━ 366s 9ms/step - accuracy: 0.7723 - loss: 0.4728 - val_accuracy: 0.7685 - val_loss: 0.4787
Epoch 4/5
40000/40000 ━━━━━━━━━━━━━━━━━━━━ 365s 9ms/step - accuracy: 0.7760 - loss: 0.4661 - val_accuracy: 0.7691 - val_loss: 0.4779
Epoch 5/5
40000/40000 ━━━━━━━━━━━━━━━━━━━━ 366s 9ms/step - accuracy: 0.7797 - loss: 0.4613 - val_accuracy: 0.7694 - val_loss: 0.4788


In [39]:
# Evaluate the model on the test data
loss, accuracy = model_1.evaluate(X_test, y_test_one_hot)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

10000/10000 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.7698 - loss: 0.4770
Test Loss: 0.47877973318099976
Test Accuracy: 0.7693687677383423


# Model2

In [40]:
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Concatenate, Flatten, Dense

embedding_dim = 100
max_sequence_length = 100

input_layer = Input(shape=(max_sequence_length,))

embedding_layer = Embedding(input_dim=v+1, output_dim=embedding_dim)(input_layer)

filter_sizes = [2, 3, 4]
convolution_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(filters=2, kernel_size=filter_size, activation='relu')(embedding_layer)
    maxpool_layer = MaxPooling1D(pool_size=max_sequence_length - filter_size + 1)(conv_layer) 
    convolution_layers.append(maxpool_layer)

concatenated_layer = Concatenate()(convolution_layers)

flatten_layer = Flatten()(concatenated_layer)

dense_layer = Dense(64, activation='relu')(flatten_layer)

output_layer = Dense(2, activation='softmax')(dense_layer)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 100, 100)  │ 48,208,600 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 99, 2)     │        402 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 98, 2)     │        602 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 97, 2)     │        802 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_6     │ (None, 1, 2)      │          0 │ conv1d_6[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_7     │ (None, 1, 2)      │          0 │ conv1d_7[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_8     │ (None, 1, 2)      │          0 │ conv1d_8[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1, 6)      │          0 │ max_pooling1d_6[… │
│ (Concatenate)       │                   │            │ max_pooling1d_7[… │
│                     │                   │            │ max_pooling1d_8[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 6)         │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │        448 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 2)         │        130 │ dense_6[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 48,210,984 (183.91 MB)

 Trainable params: 48,210,984 (183.91 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
result_4 = model.fit(X_train, y_train_one_hot, epochs=5, validation_data=(X_test, y_test_one_hot))

Epoch 1/5
40000/40000 ━━━━━━━━━━━━━━━━━━━━ 369s 9ms/step - accuracy: 0.7398 - loss: 0.5188 - val_accuracy: 0.7614 - val_loss: 0.4901
Epoch 2/5
40000/40000 ━━━━━━━━━━━━━━━━━━━━ 358s 9ms/step - accuracy: 0.7719 - loss: 0.4722 - val_accuracy: 0.7626 - val_loss: 0.4871
Epoch 3/5
40000/40000 ━━━━━━━━━━━━━━━━━━━━ 357s 9ms/step - accuracy: 0.7830 - loss: 0.4562 - val_accuracy: 0.7627 - val_loss: 0.4902
Epoch 4/5
19602/40000 ━━━━━━━━━━━━━━━━━━━━ 2:54 9ms/step - accuracy: 0.7912 - loss: 0.4421

In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# model 2 with different #of filters

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Concatenate, Flatten, Dense

embedding_dim = 100
max_sequence_length = 100

input_layer = Input(shape=(max_sequence_length,))

embedding_layer = Embedding(input_dim=v+1, output_dim=embedding_dim)(input_layer)

filter_sizes = [2, 3, 4]
convolution_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedding_layer)
    maxpool_layer = MaxPooling1D(pool_size=max_sequence_length - filter_size + 1)(conv_layer) 
    convolution_layers.append(maxpool_layer)

concatenated_layer = Concatenate()(convolution_layers)

flatten_layer = Flatten()(concatenated_layer)

dense_layer = Dense(64, activation='relu')(flatten_layer)

output_layer = Dense(2, activation='softmax')(dense_layer)

model1 = Model(inputs=input_layer, outputs=output_layer)

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model1.summary()

In [ ]:
result_5 = model1.fit(X_train, y_train_one_hot, epochs=5, validation_data=(X_test, y_test_one_hot))

# Model_3

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dropout, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

learning_rate = 0.0001  # Set your desired learning rate here
x = Embedding(v + 1, D)(inputt)
x = Dropout(0.5)(x)

# Apply convolutional layers
x = Conv1D(64, 5, activation='relu')(x)
x = MaxPooling1D(pool_size=4)(x)

x = Conv1D(32, 5, activation='relu')(x)
x = MaxPooling1D(pool_size=4)(x)

# Flatten the output of the convolutional layers
x = Flatten()(x)

x = Dense(16, activation='relu')(x)
x = Dense(2, activation='softmax')(x)

# Define the model
model_2 = tf.keras.Model(inputs=inputt, outputs=x)

# Define optimizer with specified learning rate
optimizer = Adam(learning_rate=learning_rate)

model_2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
result_2=model_2.fit(X_train, y_train_one_hot, epochs=5, validation_data=(X_test, y_test_one_hot))

# Model_4 with different kernel size

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dropout, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

learning_rate = 0.0001  # Set your desired learning rate here
x = Embedding(v + 1, D)(inputt)
x = Dropout(0.5)(x)

# Apply convolutional layers
x = Conv1D(128, 7, activation='relu')(x)
x = MaxPooling1D(pool_size=4)(x)

x = Conv1D(64, 7, activation='relu')(x)
x = MaxPooling1D(pool_size=4)(x)

# Flatten the output of the convolutional layers
x = Flatten()(x)

x = Dense(32, activation='relu')(x)
x = Dense(2, activation='softmax')(x)

# Define the model
model_3 = tf.keras.Model(inputs=inputt, outputs=x)

# Define optimizer with specified learning rate
optimizer = Adam(learning_rate=learning_rate)

model_3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
result_3=model_3.fit(X_train, y_train_one_hot, epochs=5, validation_data=(X_test, y_test_one_hot))